### Classification of ag plastics in Oxnard, CA

Annie Taylor, June 2024

Sentinel-2 classification of agricultural plastic use in Oxnard, CA based on Yuanyuan Tian's summer 2023 project.

Converted from the GEE javascript to the GEE python api

#### Import packages and initialize ee api

In [1]:
import numpy as np
import pandas as pd
import ee
import geemap

In [2]:
# ee.Authenticate(auth_mode='localhost')

In [3]:
# ee.Initialize()
ee.Initialize(project='ee-annalisertaylor')

#### Import shapefiles and convert to ee features

In [10]:
oxnard_bnd = geemap.shp_to_ee('data/agplastics/Oxnard.shp') 
strawberries = geemap.shp_to_ee('data/agplastics/Strawberries.shp')
bushberries = geemap.shp_to_ee('data/agplastics/Bushberries_T19.shp')
flowers = geemap.shp_to_ee('data/agplastics/Flowers_T16.shp')
peppers = geemap.shp_to_ee('data/agplastics/Pepers_T21.shp')
avocados = geemap.shp_to_ee('data/agplastics/Avocados.shp')

NameError: name 'polygons_label_mulch_hoop' is not defined

In [11]:
label_mulch_hoop = geemap.shp_to_ee('data/agplastics/label_mulch_hoop.shp')
label_nonplastic = geemap.shp_to_ee('data/agplastics/label_nonplastic.shp')
polygons = label_mulch_hoop.merge(label_nonplastic);

aoi_truck_crop = geemap.shp_to_ee('data/agplastics/T.shp')
aoi = aoi_truck_crop

#### Date range

In [12]:
start = '2019-02-01'
end = '2019-06-01'

In [13]:
m = geemap.Map()
m.add("basemap_selector")
m.addLayer(oxnard_bnd)
m.centerObject(oxnard_bnd, 12)
# m

#### Sentinel-2 data processing

##### helper functions

In [14]:
# mask out cloudy pixels from s2 images
def maskS2clouds(image):
    qa = image.select('QA60')
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
        .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

# Add NDVI band
def addNDVI(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# Add NDTI band
def addNDTI(image):
    ndti = image.normalizedDifference(['B11', 'B12']).rename('NDTI')
    return image.addBands(ndti)

# Add PGI band
def addPGI(image):
    NIR = image.select('B8')
    R = image.select('B4')
    G = image.select('B3')
    B = image.select('B2')
    pgi = NIR.subtract(R).multiply(100).multiply(R) \
      .divide(NIR.add(B).add(G).divide(3).subtract(1)) \
      .rename('PGI')
    return image.addBands(pgi)

# Add PMLI band
def addPMLI(image):
    SWIR1 = image.select('B11')
    R = image.select('B4')
    pmli = SWIR1.subtract(R) \
      .divide(SWIR1.add(R)) \
      .rename('PMLI')
    return image.addBands(pmli)

# Add RPGI band
def addRPGI(image):
    NIR = image.select('B8')
    R = image.select('B4')
    G = image.select('B3')
    B = image.select('B2')
    rpgi = B.multiply(100) \
      .divide(NIR.add(B).add(G).divide(3).subtract(1)) \
      .rename('RPGI')
    return image.addBands(rpgi)



##### build the collection

In [15]:
processedCollection = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterDate(start, end) \
    .filterBounds(aoi) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .map(maskS2clouds) \
    .map(addNDVI) \
    .map(addNDTI) \
    .map(addPGI) \
    .map(addPMLI) \
    .map(addRPGI)

In [17]:
# Get the median image from the processed collection and clip it to the AOI
image = processedCollection.median().clip(aoi).clip(oxnard_bnd);

# Select which bands to use for plastic presence prediction
# ref: Combining Multi-Source Data and Feature Optimization for Plastic-Covered Greenhouse Extraction and Mapping Using the Google Earth Engine : A Case in Central Yunnan
bands = ['B4', 'B3', 'B2', 'B6', 'B8', 'B11', 'B12', 'NDVI', 'NDTI', 'PGI', 'PMLI']

